In [7]:
import os
root=r'C:\Users\14833\Desktop\process_dataset\glioma_tumor'
for i,file in enumerate(os.listdir(root)[0:]):
    # print(file)
    new='glioma_tumor'+ str(i).zfill(3)+'.jpg'
    file_path=os.path.join(root,file)
    os.rename(file_path,os.path.join(root,new))
# glioma_tumor 0
# meningioma_tumor 1
# no_tumor  2
# pituitary_tumor  3

In [1]:
import torch
torch.cuda.is_available()

True

In [4]:
import os
from PIL import Image

def analyze_image_sizes(folder_path):
    image_sizes = {}
    
    # 使用os.walk递归地遍历文件夹及其子文件夹
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            
            # 检查文件是否为图片
            if not any(file_path.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
                continue
            
            # 打开图像文件
            try:
                with Image.open(file_path) as img:
                    width, height = img.size
                    size_key = f'{width} x {height}'
                    if size_key in image_sizes:
                        image_sizes[size_key] += 1
                    else:
                        image_sizes[size_key] = 1
            except (IOError, OSError):
                continue
    
    return image_sizes

# 指定要分析的文件夹路径
folder_path = r'C:\Users\14833\Desktop\archive\Testing'

# 分析图片尺寸
image_sizes = analyze_image_sizes(folder_path)

# 打印每种尺寸的图片数量
for size, count in image_sizes.items():
    print(f'{size}: {count}张')
print(len(image_sizes.keys()))   #统计有多少中尺寸

512 x 512: 787张
440 x 278: 1张
257 x 307: 1张
228 x 221: 13张
212 x 237: 2张
528 x 581: 1张
295 x 394: 1张
219 x 234: 1张
262 x 308: 1张
206 x 249: 1张
530 x 526: 1张
455 x 500: 2张
200 x 223: 2张
472 x 546: 1张
441 x 427: 1张
540 x 568: 1张
507 x 605: 1张
290 x 339: 1张
591 x 650: 1张
534 x 674: 1张
300 x 345: 1张
306 x 306: 1张
374 x 370: 1张
201 x 207: 1张
291 x 340: 1张
630 x 423: 1张
318 x 354: 1张
554 x 554: 3张
315 x 341: 1张
286 x 356: 1张
278 x 306: 1张
252 x 303: 1张
442 x 442: 9张
312 x 401: 1张
300 x 412: 1张
205 x 251: 1张
300 x 258: 1张
204 x 249: 1张
239 x 224: 1张
323 x 342: 1张
290 x 342: 1张
366 x 395: 1张
416 x 395: 1张
256 x 256: 5张
402 x 396: 1张
369 x 395: 1张
343 x 367: 1张
369 x 398: 1张
313 x 393: 1张
330 x 355: 1张
367 x 395: 1张
411 x 396: 1张
234 x 216: 1张
232 x 261: 1张
645 x 439: 1张
314 x 195: 1张
217 x 232: 1张
352 x 322: 1张
216 x 216: 1张
257 x 248: 1张
174 x 230: 1张
206 x 245: 1张
1149 x 1019: 1张
722 x 690: 1张
315 x 315: 1张
351 x 398: 1张
341 x 395: 2张
341 x 372: 1张
273 x 326: 1张
239 x 248: 1张
289 x 354: 1张
3

In [11]:
import torch
import torch.nn as nn

class MixedConvModule(nn.Module):
    def __init__(self, in_channels=1, out_channels=64):
        super(MixedConvModule, self).__init__()

        # 深度可分离卷积
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, 
              stride=1, padding=0), 
            nn.BatchNorm2d(out_channels), 
            nn.ReLU(inplace=True)
        )

        # 普通卷积
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        # 膨胀卷积
        self.dilated_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            
        )

    def forward(self, x):
        out_depthwise = self.depthwise_conv(x)
        out_conv = self.conv(x)
        out_dilated = self.dilated_conv(x)

        # 将三个卷积结果在通道维度上拼接
        out = torch.cat([out_depthwise, out_conv, out_dilated], dim=1)
        return out_depthwise,out_conv,out_dilated
inputs = torch.rand((1,1, 224, 224))
model = MixedConvModule()
# model
out1,out2,out3 = model(inputs)
print(out1.shape,out2.shape,out3.shape)


torch.Size([1, 64, 224, 224]) torch.Size([1, 64, 224, 224]) torch.Size([1, 64, 224, 224])


In [55]:
from torchvision import models
model = models.resnet50( pretrained=True)
# model = models.resnet50( )
# for i in model.parameters():   # Returns an iterator over module parameters.
#     print(i.shape)  
# for i,j in model.named_parameters():  #Returns an iterator over module parameter's name (比如conv1.weight)and parameters.
#     print(i)
    # print(j)


# for i in model.modules():   # Returns an iterator over all modules in the network.
#     print(i) 
# for i,j in model.named_modules():   # 返回一个由'module'和 'module的名字'组成的迭代器
#     print(i) 
# print(model.layer1.0.conv1)   #只能点一次如:model.layer1。model.layer1.0.conv1报错


# for i in model.state_dict(): #返回的是一个Python字典对象，它包含了模型中所有可学习参数的名称和对应的张量值,现在没load权重进去，它是空的
#     print(i)  

ckpt=torch.load(r'runs\exp\train\resnet50_attention\last.pt') 
# print(type(ckpt))
# for i in ckpt.keys():
#     print(i)

# model_state_dict = ckpt['model']   ###!!!!注意：保存要和加载配套使用，以什么形式保存，就以什么形式加载
# model.load_state_dict (model_state_dict)#这是一个方法，用于加载网络的状态字典。你可以使用这个方法将预训练模型的参数加载到网络中。
#具体来说，model.state_dict() 返回的字典包含了模型中的所有层和对应的参数。
# 每个参数都用其名称作为键，对应的张量作为值。这些参数张量保存了模型在训练过程中学到的权重和偏置等信息。


for name, child in model.named_children():
    # print(name)
    # print(child)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, t

In [ ]:
import pandas as pd

a=pd.read_csv(r'.\runs\exp\train\resnet50_attention\train.csv')  #默认sep是逗号
a
# print('====>索引{}',a['test_mean_acc'].max())    ## .max()返回最大值 ，idmax() 返回最大值对应的索引，会有一两行的偏差,与带头不带头从哪行开始有关
# print('====>索引{}',a['test_mean_acc'].idxmax())
#  max(a[a['c']])


In [ ]:
#split_data.py 划分训练集验证集
#   ！！！！划分前要求的目录结构：所以的图片和txt标签要求在总的origin_data_root目录下

import os
from shutil import move, rmtree
import random
from tqdm import tqdm

def mk_file(file_path: str):
    if os.path.exists(file_path):
        # 如果文件夹存在，则先删除原文件夹在重新创建
        rmtree(file_path)
    os.makedirs(file_path)


def split_train_val_test(origin_data_root,split_test_rate):
    # 保证随机可复现
    random.seed(0)

    # 指向你解压后的flower_photos文件夹
    # cwd = os.getcwd()
    # data_root = os.path.join(cwd, "flower_data")
    # origin_flower_path = os.path.join(data_root, "flower_photos")
    # assert os.path.exists(origin_flower_path)

    # flower_class = [cla for cla in os.listdir(origin_flower_path)
    #                 if os.path.isdir(os.path.join(origin_flower_path, cla))]

    # 建立保存训练集的文件夹
    # train_root = os.path.join(origin_data_root, "train")
    # train_img_root=os.path.join(train_root,"images")
    # train_label_root=os.path.join(train_root,"labels")
    # mk_file(train_root)
    # mk_file(train_img_root)
    # mk_file(train_label_root)

    # # 建立保存验证集的文件夹
    # val_root = os.path.join(origin_data_root, "val")
    # val_img_root=os.path.join(val_root,"images")
    # val_label_root=os.path.join(val_root,"labels")
    # mk_file(val_root)
    # mk_file(val_img_root)
    # mk_file(val_label_root)

    #tag：（test）建立保存测试集的文件夹
    # test_root = os.path.join(origin_data_root, "test")
    # test_img_root=os.path.join(test_root,"images")
    # test_label_root=os.path.join(test_root,"labels")
    # mk_file(test_root)
    # mk_file(test_img_root)
    # mk_file(test_label_root)


    images= [i for i in os.listdir(origin_data_root) if i.endswith('.jpg')]
    # print(images)
    num=len ( [i for i in os.listdir(origin_data_root) if i.endswith('.jpg')] )
    # val_index = random.sample(images, k=int(num*split_val_rate))
    # print(val_index)
    test_index=  random.sample(images, k=int(num*split_test_rate))
    # print(test_index)

    for index, image in tqdm(enumerate(images)): 
            # if image in val_index:
            #         # 将分配至验证集中的文件复制到相应目录
            #         image_path = os.path.join(origin_data_root, image)
            #         image_new_path = os.path.join(val_img_root, image)

            #         label = image.replace('.jpg','.txt')
            #         label_path= os.path.join(origin_data_root, label)
            #         label_new_path= os.path.join(val_label_root, label)


            #         move(image_path, image_new_path)
            #         move(label_path, label_new_path)

            #tag:（test）若要使用测试集打开
            if image in test_index:
                    # 将分配至验证集中的文件复制到相应目录
                    image_path = os.path.join(origin_data_root, image)
                    image_new_path = os.path.join(test_img_root, image)

                    label = image.replace('.jpg','.txt')
                    label_path= os.path.join(origin_data_root, label)
                    label_new_path= os.path.join(test_label_root, label)


                    move(image_path, image_new_path)
                    move(label_path, label_new_path)
            else :
                    # 将分配至验证集中的文件复制到相应目录
                    image_path = os.path.join(origin_data_root, image)
                    image_new_path = os.path.join(train_img_root, image)

                    label = image.replace('.jpg','.txt')
                    label_path= os.path.join(origin_data_root, label)
                    label_new_path= os.path.join(train_label_root, label)



                    move(image_path, image_new_path)
                    move(label_path, label_new_path)

            print("\r[{}] processing [{}/{}]".format(image, index+1, num), end="")  # processing bar




if __name__ == '__main__':
    origin_data_root=r"C:\Users\14833\Desktop\yolov5-6.1_official\datasets\garbage"
    # 将数据集中10%的数据划分到验证集中
    # split_val_rate = 0.15
    split_test_rate = 0.15
    split_train_val_test(origin_data_root,split_test_rate)

In [9]:
import os,random,shutil
root=r'C:\Users\14833\Desktop\process_dataset\dataset'
des=r'C:\Users\14833\Desktop\process_dataset\dataset\test'
file_paths=[]
split_test_rate=0.15
for roots,dirs,files in os.walk(root):
    # for root in roots:
    for file in files:
        file_path=os.path.join(roots,file)
        file_paths.append(file_path)
test_index=  random.sample(file_paths, k=int(len(file_paths)*split_test_rate))
len(test_index)
for index,file in enumerate(file_paths):
    if file in test_index:
        image_new_path = os.path.join(des, file.split('\\')[-2]+'\\'+file.split('\\')[-1])
        print(image_new_path)
        shutil.move(file,image_new_path)
        

# len(file_paths)


C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor001.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor002.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor005.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor007.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor017.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor021.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor024.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor032.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor036.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor038.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\glioma_tumor039.jpg
C:\Users\14833\Desktop\process_dataset\dataset\test\glioma_tumor\

In [11]:
import pandas as pd
import numpy as np
a=pd.read_csv('.\Datasets\label_7023_brain_val.csv',sep=' ',names=['a','b','c','d','e'])
# a[a['d']==1]
a['f']=a['d']    #像读取的pandas添加相同的一列  
a['f'] = np.where(a['d']== 1, 1,0 )   ##df['new_column'] = np.where(df['column_name'] > threshold, 1, 0)
a.to_csv('.\Datasets\label_7023_brian_val_CUXI.csv',sep=' ',header=None,index=None)  #不要header，index，否则会被写入文件


In [23]:
import pandas as pd
import numpy as np
a=pd.read_csv('.\Datasets\label_7023_brian_train.csv',sep=' ',names=['a','b','c','d','e'])
# a[a['d']==1]
temp=a['d']    #像读取的pandas添加相同的一列 
a['d']=a['e'] 
a['e']=temp
a['e'] = np.where(a['e']== 0, 0 , 1 )   ##df['new_column'] = np.where(df['column_name'] > threshold, 1, 0)
a.to_csv('.\Datasets\label_7023_brian_train_CUXI.csv',sep=' ',header=None,index=None)  #不要header，index，否则会被写入文件
# a

In [3]:
import torch 
if __name__ == '__main__':
    ckpt = torch.load(r'runs\exp\train\efficientnet_b0_ECA_CBAM_CAM(laeblsoooth0.1)_3064增强大数据集\best.pt')
    for key in ckpt['model'].keys():
        print(key)


total_ops
total_params
_conv_stem.weight
_conv_stem.total_ops
_conv_stem.total_params
_bn0.weight
_bn0.bias
_bn0.running_mean
_bn0.running_var
_bn0.num_batches_tracked
channel_attention1.total_ops
channel_attention1.total_params
channel_attention1.f1.weight
channel_attention1.f2.weight
channel_attention1.sigmoid.total_ops
channel_attention1.sigmoid.total_params
spatial_attention1.total_ops
spatial_attention1.total_params
spatial_attention1.conv.weight
spatial_attention1.sigmoid.total_ops
spatial_attention1.sigmoid.total_params
_blocks.total_ops
_blocks.total_params
_blocks.0.total_ops
_blocks.0.total_params
_blocks.0._depthwise_conv.weight
_blocks.0._depthwise_conv.total_ops
_blocks.0._depthwise_conv.total_params
_blocks.0._bn1.weight
_blocks.0._bn1.bias
_blocks.0._bn1.running_mean
_blocks.0._bn1.running_var
_blocks.0._bn1.num_batches_tracked
_blocks.0.conv.weight
_blocks.0.sigmoid.total_ops
_blocks.0.sigmoid.total_params
_blocks.0._project_conv.weight
_blocks.0._project_conv.total_ops